In [1]:
using DifferentialEquations
using ParameterizedFunctions
using Plots

type SSNParam
    w
    h
    τ
    V_rest
    k
    n
    η
end

ReLU(x) = x < 0.0 ? 0.0 : x

function ssn_ode(t, u, param, du)
    w = param.w
    h = param.h
    τ = param.τ
    V_rest = param.V_rest
    k = param.k
    n = param.n
    η = param.η
    du .= (((- u .+ V_rest) .+ w * (k .* ReLU.(u .- V_rest).^n) .+ h) ./ τ) + η
end

ssn_ode (generic function with 1 method)

In [2]:
w = [1.25 -.65
    1.2 -0.5]

2×2 Array{Float64,2}:
 1.25  -0.65
 1.2   -0.5 

In [3]:
#τ = [1, 0.1] #This gives unstable behavior
τ = [0.2, 0.1]

2-element Array{Float64,1}:
 0.2
 0.1

In [4]:
h = [0.0, 0.0]

2-element Array{Float64,1}:
 0.0
 0.0

In [ ]:
#Noise term in covariance matrix
type noiseParam
    s_0
    τ
    δ
end

function noise_cov(t, σ_a, param, Σ_noise)
    

In [ ]:
# Input noise Std
σ_0E = 0.2;     # input noise std. for E cells
σ_0I = 0.1;     # input noise std. for I cells
σ_0 = [s_0E; s_0I]; 

In [ ]:
# Membrane time constant
τ_E = 20; #ms; membrane time constant (20ms for E)
τ_I = 10; #ms; membrane time constant (10ms for I)
τ = [(τ_E/100); (τ_I/100)];

In [ ]:
# Covariance matrix

# step 1: create noise amplitude for E and I to model the variance of the noise
σ_a = σ_0.*sqrt(1+(tau/tau_noise));

In [ ]:
#step 2: add the noise amplitude to the noise term Sigma^(noise)
# d_ij =1 if i = j (feedforward?) and 0 otherwise (recurrent?)
# as W = [w_EE w_EI; w_IE w_II]; , set EE and II to 0 and EI and IE to 1
δ = [0 1; 1 0];

Σ_noise = σ_a.^2.*δ;

In [ ]:
#Wiener process
Wien = WienerProcess(0.0,0.0,0.0) #Wiener process starts at 0.0 at time 0.0
prob = SDEProblem(oup, u0, (0.0, 1.0), noise = Wien)
sol = solve(prob, SRIW1())

In [ ]:
#Ornstein-Uhlenbeck process for input noise
type UOP_sdeParam
    τ_noise
    cov
    Wien
    dη
    s_0
    τ
end

function oup(t, η, param, dη)
    τ_noise = param.τ_noise
    cov = param.cov
    Wien = param.Wien
    dη = (-η*dt + sqrt(2 * τ_noise * cov) * dWien)/ τ_noise
end

In [ ]:
#Time constant
τ_noise = 50;     # noise correlation time constant

In [ ]:
ssn = ParameterizedFunction(ssn_ode, SSNParam(
    w,
    h,    
    τ,
    -70.0,   # V_rest
    0.3,  # k
    2   # n
))
# add oup sde

In [ ]:
#ode = ODEProblem(ssn, ones(2), (0.0, 5.0))
ode = ODEProblem(ssn, [-80.0, 60.0], (0.0, 2.0))

In [ ]:
sol = solve(ode, verbose=true);

In [ ]:
plot(sol, vars=[1, 2])

In [ ]:
workspace() 